In [175]:
import win32com.client
import os
import numpy as np
import time
from os import getcwd
from os.path import join, dirname, exists
from math import pi, sqrt
from scipy.io import loadmat
import re
import matplotlib.pyplot as plt
# # path='D:/KangDH/Optislang_Motorcad/IPM_HDEV_2/MCAD/HDEV_Thesis2/DutyCycleData/'
# # ext_Duty_Cycle='OP1_temp_rise'
# # ext_Duty_Cycle_full=join(path,ext_Duty_Cycle)+'.dat'
# # ext_Duty_Cycle_full

# # power_filename=ext_Duty_Cycle+'_power.csv'
# # power_filename
# # from MotorCAD_Methods import MotorCAD
# mcApp = win32com.client.Dispatch("MotorCAD.AppAutomation")  # Launch Motor-CAD application

# # mc=MotorCAD()
ext_Duty_Cycle='OP1_temp_rise'
# %reset_selective -f ^(?!mcApp).*$

fun_temp_rise_external_dutycyle

In [189]:
def fun_temp_rise_external_dutycyle(path,ext_Duty_Cycle):
    ext_Duty_Cycle_full=join(path,ext_Duty_Cycle,'.dat')
    mcapp.LoadDutyCycle(ext_Duty_Cycle_full)
    mcapp.SetVariable('InitialTransientTemperatureOption',3)
    mcapp.CalculateDutyCycle_Lab()

fun_temp_rise_external_dutycyle()

In [ ]:


ext_Duty_Cycle='OP1_temp_rise'

def fun_load_temp_rise_2csvfile(lab_transient_fullpath_w_filename):
    Temp_filename=lab_transient_fullpath_w_filename+'_temp.csv'
    power_filename=lab_transient_fullpath_w_filename+'_power.csv'
    mcApp.ExportResults('Transient', power_filename)
    mcApp.ExportResults('Transient', Temp_filename)
    
fun_load_temp_rise_2csvfile(ext_Duty_Cycle)

Duty Cycle Coupled Transient 해석 결과로 나오는 mat파일 이름 변경


In [176]:
def fun_rename_matfile_lab_duty(ext_Duty_Cycle):
    ex, motpath=mcApp.GetVariable("CurrentMotFilePath_MotorLAB")
    motpath=re.sub(".mot","",motpath)
    Lab_path=motpath+'/Lab/'
    os.chdir(Lab_path)
    rename_matfile=ext_Duty_Cycle+'_lab_result.mat'
    if os.path.exists('MotorLAB_drivecycledata.mat'):
        os.rename('MotorLAB_drivecycledata.mat',rename_matfile)
        
# fun_rename_matfile_lab_duty('OP1_temp_rise')

# ext_Duty_Cycle='OP1_temp_rise'

# MotorLAB_drivecycledata




fun_load_matfile - mat 파일에서 온도에 해당하는 정보 뽑아오기

In [185]:
def fun_load_matfile(ext_Duty_Cycle):
    ex, motpath=mcApp.GetVariable("CurrentMotFilePath_MotorLAB")
    motpath=re.sub(".mot","",motpath)
    Lab_path=motpath+'/Lab/'
    os.chdir(Lab_path)
    rename_matfile=ext_Duty_Cycle+'_lab_result.mat'
    Mat_File_Data=loadmat(rename_matfile)
    return Mat_File_Data

OP1_temp_rise_mat=fun_load_matfile('OP1_temp_rise')
AG_Temp=OP1_temp_rise_mat['Airgap_Temp']
AG_Temp=AG_Temp.flatten().tolist()

In [ ]:
OP1_temp_rise_temp_dic=dict(zip(OP1_temp_rise_temp_key,OP1_temp_rise_temp))
key=OP1_temp_rise_temp_dic.keys()
max(OP1_temp_rise_temp_dic.get('Magnet_Temp'))

fun_check_temp_rise_allcomponent

In [178]:
def fun_check_temp_rise_allcomponent(ext_Duty_Cycle):
    ## init
    init_final_temp=[]
    ## change name of mat file
    fun_rename_matfile_lab_duty(ext_Duty_Cycle)
    ## load mat file
    list_from_mat=fun_load_matfile(ext_Duty_Cycle)
    
    ## 
    mat_temp=[list_from_mat.get(key) for key in list_from_mat.keys() if 'Temp' in key]
    mat_temp_key=[key for key in list_from_mat.keys() if 'Temp' in key]

    ## check temp rise
    for i in range(len(mat_temp)):
        temp=mat_temp[i].ravel().tolist()
        init_final_temp.append((temp[0],temp[-1]))
        check_temp=max(max(init_final_temp))
    
    ## 
    max_temp_key=[key for key in mat_temp_key if max(list_from_mat.get(key))==check_temp]
    dic_init_final_temp=dict(zip(mat_temp_key,init_final_temp))
    
    return dic_init_final_temp, max_temp_key, check_temp

a, b,c =check_temp_rise_allcomponent('OP1_temp_rise')

In [ ]:
## Dict to list (which is array)

mat_temp=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
mat_time=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Time' in key]
mat_temp_key=[key for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
mat_torque=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Shaft_Torque' in key]
mat_emtorque=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Electromagnetic_Torque' in key]
mat_current=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Stator_Current_Line_Peak' in key]
mat_voltage=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Voltage_Phase_Peak' in key]
mat_Terminal_Power=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Terminal_Power' in key]
mat_Perror=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Power_Error' in key]


# [Plot]온도시험 결과값 확인, 입력, 출력, 온도 - with list data (which is ndarray type)


In [ ]:

from matplotlib import legend

fig=plt.figure(figsize=(10,20))
time=mat_time[0].ravel().tolist()
for i in range(len(mat_temp)):
    temp=mat_temp[i].ravel().tolist()
    label=mat_temp_key[i]
    plt.subplot(4,1,1)
    plt.annotate(label+str(round(max(temp),2))+'degC',xy=(time[-1],temp[-1]))
    plt.plot(time,temp,label=label)
    plt.title('Temp Rise')
    plt.xlabel('Time(sec)')
    plt.ylabel('Temperature(degC)')
    

plt.legend()
plt.subplot(4,1,2)
plt.plot(time,mat_torque[0].ravel().tolist(), label='Shaft_Torque')
plt.plot(time,mat_emtorque[0].ravel().tolist(),label='Electromagnetic_Torque')
plt.legend()

plt.subplot(4,1,3)
plt.plot(time,mat_current[0].ravel().tolist(),label='Stator_Current_Line_Peak',)
plt.ylim(750,800)
plt.legend()

plt.subplot(4,1,4)
plt.plot(time,mat_voltage[0].ravel().tolist(),label='Voltage_Phase_Peak',)
plt.legend()
# plt.plot(OP1_temp_rise_time[0],OP1_temp_rise_temp[0][1])
fig.savefig(join(ext_Duty_Cycle+'png'))

In [ ]:
init_temp=temp[0]
final_temp=temp[-1]

In [ ]:
list_from_mat=fun_load_matfile(ext_Duty_Cycle)


In [ ]:
mat_temp=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]

def check_temp_rise_allcomponent(ext_Duty_Cycle):
    ## init
    init_final_temp=[]
    
    ## load mat file
    list_from_mat=fun_load_matfile(ext_Duty_Cycle)
    ## check temp rise
    for i in range(len(list_from_mat)):
        temp=list_from_mat[i].ravel().tolist()
        init_final_temp.append((temp[0],temp[-1]))
        check_temp=max(max(init_final_temp))
    return init_final_temp, check_temp

init_final_temp, check_temp=check_temp_rise_allcomponent(mat_temp)


        
    

In [ ]:
def fun_check_init_final_temp(ext_Duty_Cycle):
    Mat_File_Data=fun_load_matfile(ext_Duty_Cycle)
    mat_temp=[Mat_File_Data.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
    mat_temp_key=[key for key in OP1_temp_rise_mat.keys() if 'Temp' in key]

    Mat_File_temp_key=Mat_File_Data.keys()

    max_temp=max(Mat_File_temp)
    min_temp=min(Mat_File_temp)
    return max_temp, min_temp

fun_check_init_final_temp('OP1_temp_rise')

In [ ]:

# Data for plotting
OSL_DESIGN_NAME = 'Design0001'
fig, ax = plt.subplots()
ax.plot(RotorPosition, CoggingTorqueVW)
screenname='CoggingTorqueVW'
ax.set(xlabel='Rotor Position (Mdeg)', ylabel='CoggingTorqueVW [Nm]',
       title='CoggingTorqueVW')
ax.grid()
wdir='Z:/Thesis/Motorcad_Optislang/IPM_HDEV_1/OPD\HDEV_Thesis1.opd/HDEV__Thesis1/'
savedir=join(wdir, 'Design0001' + '/' +  '_Pic_' + screenname )
fig.savefig(join(savedir+'png'))
plt.show()
data_array=[np.transpose(RotorPosition), np.transpose(CoggingTorqueVW)]
io.savemat(join(savedir+'.mat'),{"CoggingTorqueVW":data_array})
# OP1_temp_rise_mat.keys()

Save CSV filed Manually

In [ ]:


def fun_load_temp_rise_2csvfile(lab_transient_fullpath_w_filename):
    mcApp.ExportResults('Transient', power_filename)
    Temp_filename=ext_Duty_Cycle+'_temp.csv'
    power_filename=ext_Duty_Cycle+'_power.csv'

def fun_load_temp_rise_from_MCgraph()
    temp_rise=open(Temp_filename,'r')



# Lab Peak performance function

In [ ]:
OP_list=[]

def fun_find_current_for_required_torque(ext_Duty_Cycle):
    mcApp.SetVariable('TurnsCalc_MotorLAB', i_Turns_Coil)            # Turns per coil
    mcApp.SetVariable("LabThermalCoupling", 0)                         # Coupling with Thermal
    mcApp.SetVariable("OpPointSpec_MotorLAB", 0)                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("SpeedDemand_MotorLAB", p_Speed_Peak_Array[0])                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("TorqueDemand_MotorLAB", 1300)                       # 0- Torque 4-Max temperature definition  
    mcApp.CalculateOperatingPoint_Lab()                             # Operating point calculation
    